In [1]:
import pandas as pd
import json
import psycopg2
from tqdm import tqdm

In [3]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [4]:
tour = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/tourist_attraction_with_rating_sort_by_type.csv",index_col = 0)

In [5]:
tour = tour[['title',  'tag','인기점수','contentscd.label']]
food = tour.loc[tour['contentscd.label']=='음식점'].sort_values(by='인기점수',ascending=False).reset_index(drop=True)
accom = tour.loc[tour['contentscd.label']=='숙박'].sort_values(by='인기점수',ascending=False).reset_index(drop=True)

In [11]:
accom.shape

(922, 4)

In [12]:
food.shape

(1623, 4)

In [6]:
accom['id'] = [i for i in range(1,923)]
food['id'] = [i for i in range(1,1624)]
accom = accom[['id','title','tag']]
food = food[['id','title','tag']]

In [14]:
accom.head(2)

,id,title,tag
0,1,제주조천스위스마을게스트하우스,"숙박,숙소,게스트하우스,마을관광,올레길,식당"
1,2,해비치호텔앤드리조트,"휴식/치유,5성급호텔,호텔,숙소,리조트,가족호텔,공공와이파이존,양식레스토랑,수영장,..."


In [15]:
food.head(2)

,id,title,tag
0,1,제주김만복(만복이네김밥집),"전복김밥,통전복주먹밥,미역국,음식,해물라면,라면"
1,2,봄날카페,"카페,커피,음료,음식,콜드브루,아메리카노,바닐라라떼,카페라떼,아인슈페너,카푸치노,카..."


In [9]:
accom_except_tags = ['모텔','펜션','휴양펜션','민박','고향민박','안전인증민박','농어촌민박'] + ['게스트하우스'] + ['호텔','관광호텔','가족호텔','휴양콘도','리조트'] + ['숙소']
accom_except_tags

['모텔',
 '펜션',
 '휴양펜션',
 '민박',
 '고향민박',
 '안전인증민박',
 '농어촌민박',
 '게스트하우스',
 '호텔',
 '관광호텔',
 '가족호텔',
 '휴양콘도',
 '리조트',
 '숙소']

In [12]:
food_except_tags = ['음식','식당','한식','중식','일식','양식','분식','카페']

In [13]:
accom['tag_list'] = accom['tag'].apply(lambda x: x.split(','))
food['tag_list'] = food['tag'].apply(lambda x: x.split(','))

accom['useful_tags'] = accom['tag_list'].apply(lambda x: list( set(x) - set(accom_except_tags) ))
food['useful_tags'] = food['tag_list'].apply(lambda x: list( set(x) - set(food_except_tags) ))

In [14]:
accom = accom[['id','title','useful_tags']]

In [15]:
food = food[['id','title','useful_tags']]

In [38]:
accom.sample()

,id,title,useful_tags
476,477,부영CC관광호텔,"[패키지관광, 골프]"


In [43]:
db.rollback()

In [46]:
food.head(3)

,id,title,useful_tags
0,1,제주김만복(만복이네김밥집),"[해물라면, 라면, 미역국, 통전복주먹밥, 전복김밥]"
1,2,봄날카페,"[바닐라라떼, 콜드브루, 커피, 아메리카노, 카페라떼, 아인슈페너, 카페모카, 카라..."
2,3,올래국수,"[고기국수, 향토음식]"


In [49]:
food[food['id']==3]

,id,title,useful_tags
2,3,올래국수,"[고기국수, 향토음식]"


In [17]:
food['tag_len'] = food['useful_tags'].apply(lambda x: len(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
food[food['tag_len']==0]['useful_tags']

274    []
Name: useful_tags, dtype: object

In [ ]:
food.loc[]

In [21]:
food.loc[food['useful_tags']==[]]

ValueError: ignored

In [16]:
food[food['tag_len']==0]

KeyError: ignored

In [62]:
food[food['tag_len']==0].values[0]

array([275, '몽키비치클럽 Monkey Beach', '폐업', 0], dtype=object)

In [34]:
db = load_db()
cursor = db.cursor()

In [39]:
db.rollback()

In [40]:
for now in tqdm(accom.values):
  id,title,tags = now[:]
  sql = f"update accommodation set tag = ARRAY{tags} where accommodation_id={id}"
  cursor.execute(sql)

db.commit() 

100%|██████████| 922/922 [02:44<00:00,  5.59it/s]


In [66]:
for now in tqdm(food.values):
  id,title,tags,tag_len = now[:]
  if id ==275:
    tags = ['폐업']
  sql = f"update restaurant set tag = ARRAY{tags} where restaurant_id={id}"
  cursor.execute(sql)
db.commit()

100%|██████████| 1623/1623 [04:50<00:00,  5.58it/s]
